This is just a little demo of how the code works. First import the module Data

In [1]:
import random
from Data import *

1 5


We get access to the Data class type, which can be used to load the data and provide many support function. We now create an instance of Data and load from a sample csv file. Finally, we normalize the value.

In [12]:
D = Data()
D.loadfile("sample.csv")
D.normalize()
D.set_costs()
D.cluster_K_means(10)
D.clustering.cluster_centers_

[0.15730337078651685, 0.10674157303370786, 0.08426966292134831, 0.1348314606741573, 0.06179775280898876, 0.10112359550561797, 0.0449438202247191, 0.07865168539325842, 0.11235955056179775, 0.11797752808988764]


array([[0.28430451, 0.20193394, 0.50324675, 0.56461708, 0.19875776,
        0.34815271, 0.32210669, 0.57412399, 0.34148265, 0.12109946,
        0.46486643, 0.54644165, 0.13595374],
       [0.77063712, 0.20303724, 0.61412891, 0.33532284, 0.41361556,
        0.72722323, 0.62491672, 0.35253227, 0.51004483, 0.4604365 ,
        0.5207531 , 0.60208213, 0.75268414],
       [0.63982456, 0.42793149, 0.57397504, 0.5532646 , 0.30869565,
        0.27931034, 0.09493671, 0.6591195 , 0.33564669, 0.73139931,
        0.11924119, 0.14456654, 0.26937708],
       [0.53366228, 0.61314229, 0.56016043, 0.54682131, 0.25951087,
        0.2295977 , 0.06733474, 0.71540881, 0.18033649, 0.37848407,
        0.20765583, 0.20253358, 0.24384807],
       [0.47511962, 0.43568092, 0.60913952, 0.58528585, 0.47628458,
        0.1984326 , 0.17932489, 0.20926244, 0.22798968, 0.4749457 ,
        0.16112343, 0.0999001 , 0.21573077],
       [0.64532164, 0.24736495, 0.5855615 , 0.38688431, 0.36352657,
        0.55842912, 0.48886

For raw access:

In [13]:
D.data[0][1]

[0.7052631578947371,
 0.2213438735177866,
 0.53475935828877,
 0.30927835051546404,
 0.33695652173913043,
 0.5620689655172414,
 0.5358649789029536,
 0.2641509433962264,
 0.4037854889589906,
 0.21501706484641633,
 0.5121951219512196,
 1.0,
 0.5399429386590585]

To get a copy of a point:

In [14]:
t = D.get(4)
print(t)

(4, [0.7973684210526317, 0.1758893280632411, 0.4919786096256683, 0.2783505154639176, 0.6086956521739131, 0.6965517241379311, 0.5970464135021097, 0.2075471698113207, 0.5331230283911672, 0.3728668941979522, 0.4959349593495936, 0.8937728937728937, 0.35805991440798857, 0])


To denote unknown values, denote it as None. For a point p with missing values, we can update the unknown feature f by calling D.update(p, f):

In [15]:
D.mask(t,10)
D.mask(t,1)
D.mask(t,5)
print("before:",t)
t, reward, done, info = D.step(t, 10)
print()
print("after:",t)
print()

before: (4, [0.7973684210526317, -1e-07, 0.4919786096256683, 0.2783505154639176, 0.6086956521739131, -1e-07, 0.5970464135021097, 0.2075471698113207, 0.5331230283911672, 0.3728668941979522, -1e-07, 0.8937728937728937, 0.35805991440798857, 0])

after: (4, [0.7973684210526317, -1e-07, 0.4919786096256683, 0.2783505154639176, 0.6086956521739131, -1e-07, 0.5970464135021097, 0.2075471698113207, 0.5331230283911672, 0.3728668941979522, 0.4959349593495936, 0.8937728937728937, 0.35805991440798857, 0.07692307692307693])



You can get the binary vector of which features are known by calling bool_feature(p):

In [ ]:
f = bool_feature(t[1])
print("bool features: ", f)
print("project p with f: ", len(proj(t[1],f)))

To partition the dataset into training and test sets, simply call D.split(r), r = ratio, which returns the test set and keeps D to be the training set

In [ ]:
D_test = D.split(0.85)

print("D length:", len(D.data))
print("test length:", len(D_test.data))

Let's use k-means as our clustering algorithm. Calling D.cluster_K_means(k) will set the Data's clustering algorithm as k-means.

In [ ]:
r = random.randint(0,130)
print("r:",r)
t = D.get(r)
p = t[1]
print("t:", t)

print(D.score(t))
for i in range(12):
    print("projecting out dim:", i)
    D.mask(t,i)
    print("score:", D.score(t))
    D.update(t,i)

Now, we compute the reward of state/action (s1,a) -> s2 as score(s1) - score(s2). This reward is positive
if score(s2) < score(s1); this is obviously desirable since higher score means "worse" state. 

In [ ]:
# set N random features None
N = 11
t = D.get(r)
for i in range(N):
    while True:
        r = random.randint(0,12)
        if not np.isnan(t[1][r]):
            break
    D.mask(t,r)
print("t:",t)
print("score:", D.score(t))

print("\nREWARDS FOR A SINGLE ACTION")
for i in range(len(t[1])):
    if isnan(t[1][i]):
        print("reward of updating feature", i)
        print("reward:", D.reward(t,i))
        print("")

If we want a list of actions to take, we can call D.actions(t), while D.next_state(state, action) will return the next state
with the action applied.

In [ ]:
print("available actions:", D.actions(t))
print("next state with first available action applied:", D.next_state(t,D.actions(t)[4]))